
# Gas Price Forecast


My Organization in Lake (You can organize as you want):

1. bucket_name is the bucket name of AWS.
2. date_file is a format of storing daily, monthly or weekly... data to landing, e.g: 20230101.

- s3://{bucket_name}/{date_file}/gas/ * .csv
- s3://{bucket_name}/{date_file}/eletricity/ * .json
- s3://{bucket_name}/{date_file}/public_transport/ *.parquet

Now at data lake / obj storage i can store to bronze in lakehouse for next improvements and sequences in lakehouse.

Now for each source i will prepare the dataset to store on bronze layer of lakehouse


## 0.0. Requirements


### 0.1. Imports

In [0]:
import pandas as pd

from pytz import timezone
from datetime import datetime

from pyspark.sql import functions as pf


### 0.2. Aux Functions

In [0]:
def get_insert_mode(table_name: str):
    excep = "None"

    if table_name == "gas":
        insert_mode = "append"
        raw_file_format = "csv"
        date_file = datetime.now(tz=timezone("America/Sao_Paulo")).strftime("%Y%m%d")

    elif table_name == "eletricity":
        pass

    return excep, date_file, insert_mode, raw_file_format


def read_file_format(
    file_format: str,
    bucket_name: str,
    table_name: str,
    date_file: str
):
    try:
        if file_format == "csv":
            df = spark.read.format(file_format) \
                    .option("header", "true") \
                    .load(f"s3a://{bucket_name}/{date_file}/{table_name}/*.{file_format}")

        elif file_format == "json":
            df = spark.read.format(file_format) \
                    .option("multiLine", "true") \
                    .load(f"s3a://{bucket_name}/{date_file}/{table_name}/*.{file_format}")

        else:
            return "FORMAT OR PATH_NOT_FOUND"

    except Exception as e:
        excep = e.desc

        return excep
    

def write_to_bronze(
    load_object
):
    df = df.withColumn("data_base", pf.lit(date_file))

    if insert_mode == "append":
        df.write.format("delta") \
            .mode(insert_mode) \
            .partitionBy("data_base") \
            .save(f"s3://{bucket_name}/gas_price_forecast/{table_name}")

    elif insert_mode == "overwrite":
        df.write.format("delta") \
            .mode(insert_mode) \
            .partitionBy("data_base") \
            .save(f"s3://{bucket_name}/gas_price_forecast/{table_name}")

    else:
        pass


def ingest_to_bronze(
    table_name: str,
    landing_bucket_name: str,
    bronze_bucket_name: str
):
    # Get Table Name Information
    excep, date_file, insert_mode, file_format = get_insert_mode(table_name)

    s = ""
    s = f'Start: {datetime.now(tz=timezone("America/Sao_Paulo"))}'
    s += f"\n{'=' * 39}\n"
    s += "File: {}\nDateCut: {}\nInsertMode: {}\nFileFormat: {}".format(table_name, date_file, insert_mode, file_format)
    s += f"\n{'=' * 39}\n"

    # Load From Bronze
    load_object  =read_file_format(file_format, landing_bucket_name, table_name, date_file)

    # Ingest to Bronze
    if type(load_object) == str:
        print(load_object)

    else:
        write_to_bronze(load_object)

    return None



### 0.3. S3 Connection

In [0]:
# secrets is not available on Community =D
#access_key = dbutils.secrets.get(scope = "aws", key = "aws-access-key")
#secret_key = dbutils.secrets.get(scope = "aws", key = "aws-secret-key")
#landing_bucket_name = dbutils.secrets.get(scope = "aws", key = "aws-bucket-landing")
#bronze_bucket_name = dbutils.secrets.get(scope = "aws", key = "aws-bucket-bronze")

In [0]:
# encoded_secret_key = secret_key.replace("/", "%2F")
# aws_bucket_name = "example-aws-bucket"
# mount_name = "mount-name"
# 
# dbutils.fs.mount(
#     f"s3a://{access_key}:{encoded_secret_key}@{aws_bucket_name}", 
#     f"/mnt/{mount_name}"
# )

In [0]:
#spark.sql("CREATE EXTERNAL LOCATION [IF NOT EXISTS] <example-location-name-aws> URL 's3://<bucket-name>/<example-path>'
#     WITH (CREDENTIAL <aws-credential-name>)")

In [0]:
# Another Way
# sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
# sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)
# sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "3")
# sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")
# sc._jsc.hadoopConfiguration().set("fs.s3a.disable.chunked.encoding", "True")
# sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
# sc._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.path.style.access", "True")
# sc._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")


## 1.0. Load to Bronze


### 1.1. Gas Price

In [0]:
table_name = "gas"
excep, date_file, insert_mode, file_format = get_insert_mode(table_name)

In [0]:
ingest_to_bronze(
    table_name,
    landing_bucket_name,
    bronze_bucket_name
)

Invalid URI: null or empty
